# Create Ship Meta Data 
Contains Ship name, homeport

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id, floor, lit, hash, abs
import pandas as pd

file_location = "./platform_csvs/PdM_Platform_Data - navy_platform_data.csv"
pandas_df = pd.read_csv(file_location)
ships = spark.createDataFrame(pandas_df)

ships = ships.withColumn("join_key", monotonically_increasing_id())

# Get total number of ships for Turbine ID assignment
num_ships = ships.select('designator').count()
status = (spark.read.format('json').load(folder_status)
          .drop('end_time')
          .drop('start_time')
          .drop('abnormal_sensor')
          .withColumn('model', lit('LM2500'))
          .withColumn('join_key', monotonically_increasing_id() % num_ships)
          )

# Assign turbine ID to ship metadata
ship_meta = ships.join(status, 'join_key').drop('join_key').withColumn('designator_id', abs(hash('designator')))

folder_ship = folder+'/ship_meta'
ship_meta.write.mode('overwrite').format('json').save(folder_ship)
cleanup(folder_ship)


## PUBSEC AMTRAK Metadata

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id, floor, lit

file_location = "./platform_csvs/PdM_Platform_Data - pubsec_platform_data.csv"
pandas_df = pd.read_csv(file_location)
ships = spark.createDataFrame(pandas_df)

ships = ships.withColumn("join_key", monotonically_increasing_id())

folder_status = folder+'/historical_turbine_status'
# Get total number of ships for Turbine ID assignment
num_ships = ships.select('designator').count()
status = (spark.read.format('json').load(folder_status)
          .drop('end_time')
          .drop('start_time')
          .drop('abnormal_sensor')
          .withColumn('model', lit('LM2500'))
          .withColumn('join_key', monotonically_increasing_id() % num_ships)
          )
# # Assign turbine ID to ship metadata
ship_meta = ships.join(status, 'join_key').drop('join_key').withColumn('designator_id', abs(hash('designator')))

folder_platform = folder+'/platform_meta'
ship_meta.write.mode('overwrite').format('json').save(folder_platform)
cleanup(folder_platform)